In [2]:
import boto3
import os
from dotenv import load_dotenv
# Connect to s3
load_dotenv()
 
s3 = boto3.client(
"s3",
aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
region_name=os.getenv("AWS_DEFAULT_REGION")
)

In [3]:
s3.download_file(
    'manga-recs',
    'raw/2026-02-04/manga_metadata.json',
    'manga_metadata.json'
)

In [4]:
import pandas as pd

df = pd.read_json("manga_metadata.json", encoding="utf-8")
df.head()

,title,tags,popularity,chapters,averageScore,startDate,endDate,genres,favourites,meanScore,isAdult,id,volumes,description
0,{'english': 'Monster'},"[{'category': 'Demographic', 'description': 'T...",99146,162.0,91,"{'month': 12, 'year': 1994}","{'month': 12, 'year': 2001}","[Drama, Mystery, Psychological, Thriller]",10023,91,False,30001,18.0,Everyone faces uncertainty at some point in th...
1,{'english': 'Berserk'},"[{'category': 'Theme-Drama', 'description': 'C...",226164,NaN,93,"{'month': 8, 'year': 1989}","{'month': None, 'year': None}","[Action, Adventure, Drama, Fantasy, Horror, Ps...",35859,93,False,30002,NaN,"His name is Guts, the Black Swordsman, a feare..."
2,{'english': '20th Century Boys'},"[{'category': 'Theme-Drama', 'description': 'C...",98643,249.0,88,"{'month': 9, 'year': 1999}","{'month': 4, 'year': 2006}","[Drama, Mystery, Psychological, Sci-Fi, Thriller]",7742,88,False,30003,22.0,"Humanity, having faced extinction at the end o..."
3,{'english': 'Yokohama Kaidashi Kikou'},"[{'category': 'Theme-Slice of Life', 'descript...",34568,144.0,86,"{'month': 4, 'year': 1994}","{'month': 2, 'year': 2006}","[Sci-Fi, Slice of Life]",3124,86,False,30004,14.0,Set hundreds of years in the future after an e...
4,{'english': 'Hajime no Ippo: Fighting Spirit!'},"[{'category': 'Theme-Game-Sport', 'description...",32791,NaN,87,"{'month': 9, 'year': 1989}","{'month': None, 'year': None}","[Action, Comedy, Drama, Sports]",3271,87,False,30007,NaN,Makunouchi Ippo has been bullied his entire li...


In [5]:
# 2/2/2026: DEAL WITH NANS

# check number of missing
df[["volumes", "chapters"]].isna().mean()

volumes     0.446759
chapters    0.372685
dtype: float64

bro how is it different LOL (volumes and chapters different amount of missing values)

Well i think its still fine, since our dataset is so large it should be okay to just impute with whatevers remaining

In [6]:
df[["volumes", "chapters"]].mean()
# 133 chapters 15 volumes is the average
for col in ["volumes", "chapters"]:
    df[col + "_missing"] = df[col].isna()
    df[col] = df[col].fillna(df[col].median())

df.head()


,title,tags,popularity,chapters,averageScore,startDate,endDate,genres,favourites,meanScore,isAdult,id,volumes,description,volumes_missing,chapters_missing
0,{'english': 'Monster'},"[{'category': 'Demographic', 'description': 'T...",99146,162.0,91,"{'month': 12, 'year': 1994}","{'month': 12, 'year': 2001}","[Drama, Mystery, Psychological, Thriller]",10023,91,False,30001,18.0,Everyone faces uncertainty at some point in th...,False,False
1,{'english': 'Berserk'},"[{'category': 'Theme-Drama', 'description': 'C...",226164,109.5,93,"{'month': 8, 'year': 1989}","{'month': None, 'year': None}","[Action, Adventure, Drama, Fantasy, Horror, Ps...",35859,93,False,30002,13.0,"His name is Guts, the Black Swordsman, a feare...",True,True
2,{'english': '20th Century Boys'},"[{'category': 'Theme-Drama', 'description': 'C...",98643,249.0,88,"{'month': 9, 'year': 1999}","{'month': 4, 'year': 2006}","[Drama, Mystery, Psychological, Sci-Fi, Thriller]",7742,88,False,30003,22.0,"Humanity, having faced extinction at the end o...",False,False
3,{'english': 'Yokohama Kaidashi Kikou'},"[{'category': 'Theme-Slice of Life', 'descript...",34568,144.0,86,"{'month': 4, 'year': 1994}","{'month': 2, 'year': 2006}","[Sci-Fi, Slice of Life]",3124,86,False,30004,14.0,Set hundreds of years in the future after an e...,False,False
4,{'english': 'Hajime no Ippo: Fighting Spirit!'},"[{'category': 'Theme-Game-Sport', 'description...",32791,109.5,87,"{'month': 9, 'year': 1989}","{'month': None, 'year': None}","[Action, Comedy, Drama, Sports]",3271,87,False,30007,13.0,Makunouchi Ippo has been bullied his entire li...,True,True


In [7]:
# 2/6/2026 TEST ON NEW FUNCTIONS ---------------------
from manga_recs.data_engineering.processing.clean import (
    clean_manga_metadata, id_quality, describe_outliers
)

df = clean_manga_metadata(df)

print(id_quality(df, "id"))
describe_outliers(df, ["chapters","volumes","popularity","favourites"])
df.head()

{'missing_ids': 0, 'duplicate_ids': 0}


,title,tags,popularity,chapters,averageScore,startDate,endDate,genres,favourites,meanScore,...,description,volumes_missing,chapters_missing,popularity_log,favourites_log,start_year,start_month,end_year,end_month,is_finished
0,Monster,"[Demographic, Theme-Drama, Theme-Other, Cast-T...",99146,162,91,"{'month': 12, 'year': 1994}","{'month': 12, 'year': 2001}","[Drama, Mystery, Psychological, Thriller]",10023,91,...,Everyone faces uncertainty at some point in th...,False,False,11.504359,9.212737,1994,12,2001,12,True
1,Berserk,"[Theme-Drama, Demographic, Cast-Main Cast, Cas...",226164,110,93,"{'month': 8, 'year': 1989}","{'month': None, 'year': None}","[Action, Adventure, Drama, Fantasy, Horror, Ps...",35859,93,...,"His name is Guts, the Black Swordsman, a feare...",True,True,12.329020,10.487378,1989,8,<NA>,<NA>,False
2,20th Century Boys,"[Theme-Drama, Cast-Main Cast, Theme-Other, Dem...",98643,249,88,"{'month': 9, 'year': 1999}","{'month': 4, 'year': 2006}","[Drama, Mystery, Psychological, Sci-Fi, Thriller]",7742,88,...,"Humanity, having faced extinction at the end o...",False,False,11.499273,8.954544,1999,9,2006,4,True
3,Yokohama Kaidashi Kikou,"[Theme-Slice of Life, Cast-Main Cast, Setting-...",34568,144,86,"{'month': 4, 'year': 1994}","{'month': 2, 'year': 2006}","[Sci-Fi, Slice of Life]",3124,86,...,Set hundreds of years in the future after an e...,False,False,10.450713,8.047190,1994,4,2006,2,True
4,Hajime no Ippo: Fighting Spirit!,"[Theme-Game-Sport, Demographic, Cast-Main Cast...",32791,110,87,"{'month': 9, 'year': 1989}","{'month': None, 'year': None}","[Action, Comedy, Drama, Sports]",3271,87,...,Makunouchi Ippo has been bullied his entire li...,True,True,10.397940,8.093157,1989,9,<NA>,<NA>,False


In [10]:
i = df["tags"].dropna().index[1]
print(i)
print(df.loc[i, "tags"])

1
['Theme-Drama', 'Demographic', 'Cast-Main Cast', 'Cast-Traits', 'Theme-Other', 'Sexual Content', 'Theme-Fantasy', 'Theme-Action', 'Setting-Scene', 'Theme-Other-Organisations', 'Setting-Time', 'Theme-Romance', 'Theme-Other-Vehicle']
